In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from  sklearn import preprocessing

In [ ]:
app_info = pd.read_csv('../data/app_info.csv',names=['appid','category'])
user_app_actived = pd.read_csv('../data/user_app_actived.csv',names=['uid','appid'])
user_basic_info = pd.read_csv('../data/user_basic_info.csv',names=['uid','gender','city','prodName','ramCapacity', 'ramLeftRation','romCapacity','romLeftRation','color', 'fontSize','ct','carrier','os'])
user_basic_info.drop(columns=['city','prodName','color','ct','carrier'],inplace=True)
age_test_data = pd.read_csv('../data/age_test.csv',names=['uid'])
age_train_data = pd.read_csv('../data/age_train.csv',names=['uid','age_group'])
age_test_data = pd.read_csv('../data/age_test.csv',names=['uid'])
age_train_data = pd.read_csv('../data/age_train.csv',names=['uid','age_group'])

In [ ]:
def mem_usage(pandas_obj):
    if isinstance(pandas_obj,pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else:
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb = usage_b/1024**2
    return "{:03.2f}MB".format(usage_mb)


def optimize_memory(df):
    for col in ['appid','use_date']:
        df[col] = df[col].astype('category')
    df['uid'] = df['uid'].astype('uint32')
    for col in ['duration','times']:
        df[col] = df[col].astype('float32')
    return df

def get_use_eight_message(name,num):
    data = pd.read_csv('../data/user_app_usage_feature_reset.csv')
    data = data.sort_values(by=[name+'_mean'],ascending=False)
    app = data.groupby(['uid'])['appid'].agg('#'.join)
    app = app.reset_index()
    data[name+'_mean'] = data[name+'_mean'].apply(lambda x:str(x))
    app_mean_value = data.groupby(['uid'])[name+'_mean'].agg(' '.join)
    app_mean_value = app_mean_value.reset_index()

    data[name+'_max'] = data[name+'_max'].apply(lambda x:str(x))
    app_max_value = data.groupby(['uid'])[name+'_max'].agg(' '.join)
    app_max_value = app_max_value.reset_index()

    data[name+'_min'] = data[name+'_min'].apply(lambda x:str(x))
    app_min_value = data.groupby(['uid'])[name+'_min'].agg(' '.join)
    app_min_value = app_min_value.reset_index()

    del data
    app['appidlist'] = app['appid'].apply(lambda x:x.split('#')[0:15])
    app_mean_value[name+'_mean'] = app_mean_value[name+'_mean'].apply(lambda x:x.split(' ')[0:15])
    app_max_value[name+'_max'] = app_max_value[name+'_max'].apply(lambda x:x.split(' ')[0:15])
    app_min_value[name+'_min'] = app_min_value[name+'_min'].apply(lambda x:x.split(' ')[0:15])


    for i in range(15):
        app['appid'+str(i+num)] = app['appidlist'].apply(lambda x:x[i] if i<len(x) else np.nan)
    app.drop(columns=['appidlist'],inplace=True)
    for i in range(15):
        app_mean_value['app_mean_value'+str(i+num)] = app_mean_value[name+'_mean'].apply(lambda x:float(x[i]) if i<len(x) else np.nan)
    for i in range(15):
        app_max_value['app_max_value'+str(i+num)] = app_max_value[name+'_max'].apply(lambda x:float(x[i]) if i<len(x) else np.nan)
    for i in range(15):
        app_min_value['app_min_value'+str(i+num)] = app_min_value[name+'_min'].apply(lambda x:float(x[i]) if i<len(x) else np.nan)
    
    app_mean_value.drop(columns=[name+'_mean'],inplace=True)
    app_max_value.drop(columns=[name+'_max'],inplace=True)
    app_min_value.drop(columns=[name+'_min'],inplace=True)
    
    app = app.merge(app_mean_value,on=['uid'],how='left')
    app = app.merge(app_max_value,on=['uid'],how='left')
    app = app.merge(app_min_value,on=['uid'],how='left')
    app.to_csv('../feature/use_most_eight_{}_app.csv'.format(name),index=False)

In [ ]:
app_cate_dict = {}
appid_list = list(app_info['appid'].values)
category_list = list(app_info['category'].values)
for i in range(len(appid_list)):
    if appid_list[i] in  app_cate_dict:
        app_cate_dict[appid_list[i]].append(category_list[i])
    else:
        app_cate_dict[appid_list[i]] = []
        app_cate_dict[appid_list[i]].append(category_list[i])
user_app_num = {}
for i in user_app_actived['appid'].values:
    idlist = i.split('#')
    for j in idlist:
        if j in user_app_num:
            user_app_num[j] = user_app_num[j]+1
        else:
            user_app_num[j] = 1

In [ ]:
user_app_num2 = sorted(user_app_num.items(),key=lambda d:d[1],reverse=True)

In [ ]:
user_app_num_sort = {}
for i,j in enumerate(user_app_num2):
    user_app_num_sort[j[0]] = i+1
print(user_app_num_sort)

In [ ]:
reader = pd.read_csv('../data/user_app_usage.csv',iterator=True,names=['uid','appid','duration','times','use_date'])
chunkSize = 10000000
loop = True
allchunk = pd.DataFrame()

In [ ]:
allchunk.drop(columns=['use_date'],axis=1,inplace=True)
for col in allchunk.columns:
    if col=='appid':
        continue
    _ = allchunk.groupby(['appid'],as_index=False)[col].agg({col+'_'+col.split('_')[-1]:col.split('_')[-1]})
    allchunk = allchunk.merge(_,on=['appid'],how='left')
    allchunk.drop(columns=[col],inplace=True)

In [ ]:
allchunk = allchunk.drop_duplicates(['appid'])
allchunk['number_sort'] = allchunk['appid'].apply(lambda x:user_app_num_sort[x] if x in user_app_num_sort else np.nan)
allchunk['app_cate'] = allchunk['appid'].apply(lambda x:app_cate_dict[x][0] if x in app_cate_dict else 'unknow')
allchunk.head(10)

In [ ]:
allchunk.to_csv('../feature/app_feature.csv',index=False)

In [ ]:
loop = True
allchunk = pd.DataFrame()
while loop:
    try:
        chunk = reader.get_chunk(chunkSize)
        chunk = optimize_memory(chunk)
        chunk.drop(columns=['use_date'],axis=1,inplace=True)
        chunk['duration_average'] = chunk['duration']/chunk['times']
        print(len(chunk),mem_usage(chunk))

        for col in ['duration','times','duration_average']:
            _ = chunk.groupby(['uid','appid'],as_index=False)[col].agg({col+'_max':'max',col+'_min':'min',col+'_mean':'mean'})
            chunk = chunk.merge(_,on=['uid','appid'],how='left')
            chunk.drop(columns=[col],axis=1,inplace=True)

        print(len(chunk))
        chunk = chunk.drop_duplicates(['uid','appid'])
        print(len(chunk))
        allchunk = pd.concat([allchunk,chunk],ignore_index=True)
        print("allchunk:",len(allchunk),mem_usage(allchunk))
    except StopIteration:
        loop = False
        print("Iteration is stopped")

for col in ['duration_max','duration_min','duration_mean','times_max','times_min','times_mean',
            'duration_average_max','duration_average_min','duration_average_mean']:
    _ = allchunk.groupby(['uid','appid'],as_index=False)[col].agg({col+'_'+col.split('_')[-1]:col.split('_')[-1]})
    allchunk = allchunk.merge(_,on=['uid','appid'],how='left')
    allchunk.drop(columns=[col],inplace=True)



allchunk = allchunk.drop_duplicates(['uid','appid'])
allchunk = allchunk.rename(columns={'duration_max_max':'duration_max','duration_min_min':'duration_min','duration_mean_mean':'duration_mean',
                         'times_max_max':'times_max','times_min_min':'times_min','times_mean_mean':'times_mean',
                        'duration_average_max_max':'duration_average_max','duration_average_min_min':'duration_average_min','duration_average_mean_mean':'duration_average_mean'
                         })
print(len(allchunk))

allchunk.to_csv('../data/user_app_usage_feature_reset.csv', index=False)

In [ ]:
get_use_eight_message('duration',15)

In [ ]:
get_use_eight_message('times',0)

In [ ]:
most_dura = pd.read_csv('../feature/use_most_eight_duration_app.csv')
app_feature = pd.read_csv('../feature/app_feature.csv')

In [ ]:
age_train_data = age_train_data.merge(most_dura,on=['uid'],how='left')
age_test_data = age_test_data.merge(most_dura,on=['uid'],how='left')

In [ ]:
for i in age_train_data.columns:
    if 'appid' in i:
        age_train_data.drop(columns=[i],inplace=True)
        age_test_data.drop(columns=[i],inplace=True)
    if 'value' in i:
        age_train_data[i] = age_train_data[i].fillna(age_train_data[i].mean())
        age_test_data[i] = age_test_data[i].fillna(age_train_data[i].mean())

In [ ]:
value = [i for i in age_train_data.columns if 'value' in i]
train_weight = age_train_data[value].values
train_weight = train_weight.reshape(len(age_train_data),3,15).swapaxes(1,2)
test_weight = age_test_data[value].values
test_weight = test_weight.reshape(len(age_test_data),3,15).swapaxes(1,2)

In [ ]:
print(train_weight.shape)
print(test_weight.shape)

In [ ]:
np.save('../feature/nn_weight_train.npy',train_weight)
np.save('../feature/nn_weight_test.npy',test_weight)

In [ ]:
user_app_actived['len'] = user_app_actived['appid_2'].apply(lambda x:len(x))
print(len(user_app_actived))
user_app_actived = user_app_actived[user_app_actived['len']>5]
user_app_actived.drop(columns=['len'],inplace=True)
print(len(user_app_actived))

In [ ]:
most_dura = most_dura[['uid','appid']]

In [ ]:
age_train_data = age_train_data.merge(most_dura,on=['uid'],how='left')
age_train_data = age_train_data.merge(user_app_actived,on=['uid'],how='left')
age_test_data = age_test_data.merge(most_dura,on=['uid'],how='left')
age_test_data = age_test_data.merge(user_app_actived,on=['uid'],how='left')

In [ ]:
pp_age = {}
app_list = age_train_data['appid'].values
app_age_list = age_train_data['age_group'].values
for num,i in enumerate(app_age_list):
    app = app_list[num]
    if app is np.nan:
        continue
    if i in app_age:
        for j in app.split('#'):
            if j in app_age[i]:
                app_age[i][j] = app_age[i][j]+1
            else:
                app_age[i][j] = 1
    else:
        for j in app.split('#'):
            app_age[i] = {}
            app_age[i][j] = 1

In [ ]:
app_age_value = pd.DataFrame(app_age)
app_age_value = app_age_value.reset_index().rename(columns={'index':'appid'})
age_train_data['appid'] = age_train_data.apply(lambda x:x['appid'] if x['appid'] is not np.nan else x['appid_2'],axis=1)
age_test_data['appid'] = age_test_data.apply(lambda x:x['appid'] if x['appid'] is not np.nan else x['appid_2'],axis=1) 
age_train_data.drop(columns=['appid_2'],inplace=True)
age_test_data.drop(columns=['appid_2'],inplace=True)
age_train_data['appid'] = age_train_data['appid'].fillna('a00289791#a0048467')
age_test_data['appid'] = age_test_data['appid'].fillna('a00289791#a0048467')
age_train_data['appid'].isnull().sum()

In [ ]:
app_feature = app_feature.merge(app_age_value,on='appid',how='left')

In [ ]:
app_feature['allnum'] = app_feature[1]+app_feature[2]+app_feature[3]+app_feature[4]+app_feature[5]+app_feature[6]
for i in [1,2,3,4,5,6]:
    app_feature[i] = app_feature[i]/(app_feature['allnum']+1)

In [ ]:
for i in [1,2,3,4,5,6]:
    app_feature[i] = app_feature[i].fillna(app_feature[i].mean())

In [ ]:
app_feature['app_cate'] = app_feature['app_cate'].map(dict(zip(app_feature['app_cate'].unique(), range(0, app_feature['app_cate'].nunique()))))
for i in app_feature.columns:
    if i=='appid':
        continue
    app_feature[i] = app_feature[i].fillna(app_feature[i].mean())

In [ ]:
app_dict = dict(zip(app_feature.values[:,0],app_feature.values[:,1:]))
def get_app_feature_vec(a):
    featurevec = np.zeros([15,39])
    for num,i in enumerate(a.split('#')):
        if num>=15:
            break
        featurevec[num,:] = app_dict[i]
    return featurevec
age_train_data['appid'] = age_train_data['appid'].apply(lambda x:get_app_feature_vec(x))

In [ ]:
age_test_data['appid'] = age_test_data['appid'].apply(lambda x:get_app_feature_vec(x))

In [ ]:
train_fea = np.array(age_train_data['appid'].tolist())
test_fea = np.array(age_test_data['appid'].tolist())
np.save('../feature/nn_train.npy',train_fea)
np.save('../feature/nn_test.npy',test_fea)

In [ ]:
train = np.load('../feature/nn_train.npy')
test = np.load('../feature/nn_test.npy')
train_weight = np.load('../feature/nn_weight_train.npy')
test_weight = np.load('../feature/nn_weight_test.npy')
train_person = np.load('../feature/nn_person_train.npy')
test_person = np.load('../feature/nn_person_test.npy')
train_mat1 = np.load('../feature/nn_mat1_train.npy')
test_mat1 = np.load('../feature/nn_mat1_test.npy')
train_mat2 = np.load('../feature/nn_mat2_train.npy')
test_mat2 = np.load('../feature/nn_mat2_test.npy')

In [ ]:
train = train[0:100000]
test = test[0:100000]
train_weight = train_weight[0:100000]
test_weight = test_weight[0:100000]
train_person = train_person[0:100000]
test_person = test_person[0:100000]
train_mat1 = train_mat1[0:100000]
test_mat1 = test_mat1[0:100000]
train_mat2 = train_mat2[0:100000]
test_mat2 = test_mat2[0:100000]

In [ ]:
train_mat = np.concatenate([train_mat1],1)
test_mat = np.concatenate([test_mat1],1)
del train_mat1
del test_mat1
train_person = np.concatenate([train_person,train_mat2],1)
test_person = np.concatenate([test_person,test_mat2],1)
del train_mat2
del test_mat2

In [ ]:
train_weight = train_weight.reshape(len(train),15,3)
test_weight = test_weight.reshape(len(test),15,3)

train_person = train_person.reshape(len(train),1,-1)
test_person = test_person.reshape(len(test),1,-1)

train_mat = train_mat.reshape(len(train),1,-1)
test_mat = test_mat.reshape(len(test),1,-1)

In [ ]:
age_train_data = pd.read_csv('../data/age_train.csv',names=['uid','age_group'])
age_train_data['age_group'] = age_train_data['age_group'].apply(lambda x:x-1)
y_train = age_train_data['age_group'].values

In [ ]:
from sklearn.model_selection import StratifiedKFold
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=0)
from  sklearn import preprocessing
min_max_scaler = preprocessing.MinMaxScaler()

for i in range(train.shape[2]):
    scaler = preprocessing.StandardScaler().fit(train[:,:,i])
    train[:,:,i] = scaler.transform(train[:,:,i]) 
    test[:,:,i] = scaler.transform(test[:,:,i])
#     train[:,:,i] = min_max_scaler.fit_transform(train[:,:,i])
#     test[:,:,i] = min_max_scaler.transform(test[:,:,i])
for i in range(train_weight.shape[2]):
    scaler = preprocessing.StandardScaler().fit(train_weight[:,:,i])
    train_weight[:,:,i] = scaler.transform(train_weight[:,:,i]) 
    test_weight[:,:,i] = scaler.transform(test_weight[:,:,i])
#     train_weight[:,:,i] = min_max_scaler.fit_transform(train_weight[:,:,i])
#     test_weight[:,:,i] = min_max_scaler.transform(test_weight[:,:,i])
for i in range(train_person.shape[2]):
    scaler = preprocessing.StandardScaler().fit(train_person[:,:,i])
    train_person[:,:,i] = scaler.transform(train_person[:,:,i]) 
    test_person[:,:,i] = scaler.transform(test_person[:,:,i])

In [ ]:
from torch.utils.data import DataLoader,Dataset
class Getdata(Dataset):
    def __init__(self,feature,weight,person,mat1,labels,transform=preprocessing.normalize):
        self.feature = feature
        self.weight = weight
        self.person = person
        self.mat1 = mat1
        self.labels = labels
        self.transform = transform
    def __len__(self):
        return len(self.feature)
    def __getitem__(self, item):
        sample={}
        if self.labels is not None:
            sample['label'] = self.labels[item]
            sample['feature'] = self.transform(self.feature[item],norm='l2')
            sample['weight'] = self.transform(self.weight[item],norm='l2')
            sample['person'] = self.transform(self.person[item],norm='l2')
            sample['mat1'] = self.mat1[item]
        return sample

class Gettestdata(Dataset):
    def __init__(self,feature,weight,person,mat1,transform=preprocessing.normalize):
        self.feature = feature
        self.weight = weight
        self.person = person
        self.mat1 = mat1
        self.transform = transform
    def __len__(self):
        return len(self.feature)
    def __getitem__(self, item):
        sample={}
        if self.feature is not None:
            sample['feature'] = self.transform(self.feature[item],norm='l2')
            sample['weight'] = self.transform(self.weight[item],norm='l2')
            sample['person'] = self.transform(self.person[item],norm='l2')
            sample['mat1'] = self.mat1[item]
        return sample

In [ ]:
class Mymodel(nn.Module):
    def __init__(self,hidden_dim1=600,hidden_dim2=100,hidden_dim3=6):
        super(Mymodel,self).__init__()
        self.mat1cnn = nn.Sequential(*[
            nn.Conv2d(1,16,(1,3),stride=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.Conv2d(16,32,(1,3),stride=3),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.Conv2d(32,64,(1,5),stride=5),
            nn.BatchNorm2d(64),
            nn.ReLU(),
        ])
        self.modelperson = nn.Sequential(*[
            nn.Linear(1856, hidden_dim1),
            nn.ReLU(),
            nn.Linear(hidden_dim1, hidden_dim2),
            nn.ReLU(),
            nn.Linear(hidden_dim2, hidden_dim3),
            nn.ReLU(),
        ])
        self.personcnn = nn.Sequential(*[
            nn.Conv2d(1,16,(1,3),stride=1),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.Conv2d(16,32,(1,3),stride=3),
            nn.BatchNorm2d(32),
            nn.ReLU(),    
        ])
        self.featurecnn = nn.Sequential(*[
            nn.Conv2d(1,16,2,2),
            nn.BatchNorm2d(16),
            nn.ReLU(),
            nn.MaxPool2d(1,1),
            nn.Conv2d(16,32,3,3),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(1,1),
        ])
    def forward(self,fea_weight,person,mat1):
        fea_weight = fea_weight.float()
        person = person.float()
        person = person.unsqueeze(1)
        mat1 = mat1.float()
        mat1 = self.mat1cnn(mat1)
        mat1 = mat1.view(mat1.shape[0],-1)
        fea_weight = fea_weight.unsqueeze(1)
        fea_weight = self.featurecnn(fea_weight)
        fea_weight = fea_weight.view(fea_weight.shape[0],-1)
        person = self.personcnn(person)
        person = person.view(person.shape[0],-1)
        allfeature = torch.cat((mat1,fea_weight,person),1)
        end = self.modelperson(allfeature)
        return end

In [ ]:
device = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda")

In [ ]:
from tqdm import tqdm
from torch.autograd import Variable
from sklearn.metrics import accuracy_score
np.random.seed(1)
torch.manual_seed(1)
torch.cuda.manual_seed_all(1)
oof = np.zeros(len(train))
predictions = np.zeros(len(test))
for fold_, (trn_idx, val_idx) in enumerate(folds.split(train, y_train)):
    print(fold_)
    if fold_==1:
        break
    model = Mymodel()
    model = model.to(device)
    train_loaders = DataLoader(Getdata(train[trn_idx],train_weight[trn_idx],
                                       train_person[trn_idx],train_mat[trn_idx],y_train[trn_idx]),batch_size=64,shuffle=True)
    val_loaders = DataLoader(Getdata(train[val_idx],train_weight[val_idx],
                                      train_person[val_idx],train_mat[val_idx],y_train[val_idx]),batch_size=64,shuffle=True)
    test_loaders = DataLoader(Gettestdata(test,test_weight,test_person,test_mat),batch_size=64)
    n_ep = 4
    wd = 1e-5
    lr = 1e-4
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), weight_decay=wd, lr=lr)
    for i_ep in range(n_ep):
        for _,sample in tqdm(enumerate(train_loaders),total=len(train_loaders)):
            feature = sample['feature'].to(device)
            weight = sample['weight'].to(device)
            fea_weight = torch.cat((feature,weight),2)
            person = sample['person'].to(device)
            mat1 = sample['mat1'].to(device)
            mat1 = mat1.unsqueeze(1)
            label = sample['label'].to(device)
            optimizer.zero_grad()
            result = model.forward(fea_weight,person,mat1)
            loss = criterion(result,label)
            loss.backward()
            optimizer.step()
        with torch.no_grad():
            valmodel = model.eval()
            losses = []
            labels = []
            preds = []
            for _,sample_val in enumerate(val_loaders):
                feature_val = sample_val['feature'].to(device)
                weight_val = sample_val['weight'].to(device)
                fea_weight_val = torch.cat((feature_val,weight_val),2)
                person_val = sample_val['person'].to(device)
                mat1_val = sample_val['mat1'].to(device)
                mat1_val = mat1_val.unsqueeze(1)
                label_val = sample_val['label'].to(device)
                result_val = valmodel.forward(fea_weight_val,person_val,mat1_val)
                preds += [np.argmax(result_val.detach().cpu().numpy(),axis=1)]
                labels += [label_val.detach().cpu().numpy()]
            preds = np.concatenate(preds, 0)
            labels = np.concatenate(labels, 0)
            print(accuracy_score(labels,preds))
    with torch.no_grad():
        testmodel = model.eval()
        preds_test = []
        for _,sample_test in tqdm(enumerate(test_loaders),total=len(test_loaders)):
            feature_test = sample_test['feature'].to(device)
            weight_test = sample_test['weight'].to(device)
            fea_weight_test = torch.cat((feature_test,weight_test),2)
            person_test = sample_test['person'].to(device)
            mat1_test = sample_test['mat1'].to(device)
            mat1_test = mat1_test.unsqueeze(1)
            result_test = testmodel.forward(fea_weight_test,person_test,mat1_test)
            preds_test += [np.argmax(result_test.detach().cpu().numpy(),axis=1)]
        np = np.concatenate(preds_test, 0)
        print(predictions)

In [ ]:
age_test_data = pd.read_csv('../data/age_test.csv',names=['id'])

In [ ]:
age_test_data['label'] = predictions
age_test_data['label'] = age_test_data['label'].apply(lambda x:round(x+1))

In [ ]:
age_test_data[['id','label']].to_csv('../submit/submission.csv',index=False)